In [1]:
%run ../chap01/abalone.ipynb

In [2]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [3]:
def load_pulsar_dataset():
    with open('../../data/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    # 입력 데이터가 8개 => 갯수 맞춰서 설정
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')

In [4]:
def forward_postproc(output, y):
    # 라이브러리로 가능하다.
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    # 기타 정보들
    y, output, entropy = aux
    
    # 각 원소의 손실기울기를 부여
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)    
    
    # 시그모이드 교차 엔트로피의 편미분을 구해서,
    # entropy, output 사이의 부분 기울기를 계산
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [5]:
def eval_accuracy(output, y):
    # 시그모이드 함수 => 입력: 로짓값 / 출력: 확률
    # 0이면     => 0.5, 판단 불가!
    # 0 작으면  => 0.5 아래, 다른걸로 판단.
    # 0 크면   => 0.5 위, 같은걸로 판단.
    
    # 판단한 로짓값 ( 신경망이 추정한 로짓값에 대한 판단 )
    estimate = np.greater(output, 0)
    # 정답으로 주어진 판단
    answer = np.greater(y, 0.5)
    # 비교해서 같은 정도가 정확도로 한다.
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [6]:
# 시그모이드 관련 함수들

# relu
# x 배열의원소 각각에 대해 해당 원소값과 0 가운데 작지 않은 쪽을 골라준다.
# => 음수인 원소들을 모두 찾아내서 0으로 대치한다.
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
# 여기서는 사용 X
# 나아아아~중에 11장 LSTM에서 중요하게 사용
def sigmoid_derv(x, y):
    return y * (1 - y)


# 수식 관련 pass했다.
def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)